In [1]:
from IGraph import *
from AbstractGraph import *
from UGraphModel import *
from DGraphModel import *
#from BayesNet import *

In [180]:
import numpy as np

def uniqueElement(observation):
    # intilize a null list
    unique_list = []
    # traverse for all elements
    for x in observation:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

def get_value(factor, key):
    newd = dict()
    for i in range(len(factor)):
        newd.update({tuple([factor[i][j] for j in range(len(factor[i])-1)]):factor[i][-1]})
    return newd.get(key)

def swapCommon(factor,key,begin):
    if type(factor[0][-1]) is float:
        if begin: #swap common to the first row
            print(len(factor))
            for row in range(len(factor)):
                tmp = factor[row][0]
                factor[row][0] = factor[row][key]
                factor[row][key] = tmp
        else: # swap common to the latest row
            print(len(factor))
            for row in range(len(factor)):
                tmp = factor[row][-2]
                factor[row][-2] = factor[row][key]
                factor[row][key] = tmp
    else:
        if begin: #swap common to the first row
            print(len(factor))
            tmp = factor[0]
            factor[0] = factor[key]
            factor[key] = tmp
        else: # swap common to the latest row
            print(len(factor))
            tmp = factor[-1]
            factor[-1] = factor[key]
            factor[key] = tmp

def parsing_combinationProb(listProb):
        #print("tableValue: ", tableValue)
        output = []
        n = len(listProb)
        total = 1
        for i in range(n):
            total *= len(listProb[i])
        index = np.zeros(n)
        index[-1] = -1

        count = 0
        while count < total:
            if index[n-1] != len(listProb[n-1]) - 1:
                index[n-1] += 1
            else:
                index[n-1] = 0
                preceding = n - 1
                while True:
                    preceding -= 1
                    if index[preceding] != len(listProb[preceding]) - 1:
                        index[preceding] += 1
                        break
                    else:
                        index[preceding] = 0
            aResult = []
            for i in range(n):
                aResult.append(listProb[i][int(index[i])])
            #print("count: ", count)
            output.append(aResult)
            count += 1
        #print(output)
        return output


In [181]:
xray=[[ 'True', 'True', 0.885 ],
     [ 'True', 'False', 0.115 ],
     [ 'False', 'True', 0.04 ],
     [ 'False', 'False', 0.96 ]]
xray

[['True', 'True', 0.885],
 ['True', 'False', 0.115],
 ['False', 'True', 0.04],
 ['False', 'False', 0.96]]

In [182]:
tt = ('True','False')
print(get_value(xray,tt))

0.115


In [183]:
kaka = [[ 'Small', 'Guitar', 'A', 0.0 ],
        [ 'Small', 'Guitar', 'B', 0.5 ],
        [ 'Small', 'Guitar', 'C', 0.5 ],
        [ 'Small', 'Bass', 'A', 0.0 ],
        [ 'Small', 'Bass', 'B', 0.0 ],
        [ 'Small', 'Bass', 'C', 1.0 ],
        [ 'Small', 'Drum', 'A', 0.0 ],
        [ 'Small', 'Drum', 'B', 1.0 ],
        [ 'Small', 'Drum', 'C', 0.0 ],
        [ 'Medium', 'Guitar', 'A', 0.0 ],
        [ 'Medium', 'Guitar', 'B', 0.0 ],
        [ 'Medium', 'Guitar', 'C', 1.0 ],
        [ 'Medium', 'Bass', 'A', 0.5 ],
        [ 'Medium', 'Bass', 'B', 0.0 ],
        [ 'Medium', 'Bass', 'C', 0.5 ],
        [ 'Medium', 'Drum', 'A', 1.0 ],
        [ 'Medium', 'Drum', 'B', 0.0 ],
        [ 'Medium', 'Drum', 'C', 0.0 ],
        [ 'Big', 'Guitar', 'A', 0.0 ],
        [ 'Big', 'Guitar', 'B', 1.0 ],
        [ 'Big', 'Guitar', 'C', 0.0 ],
        [ 'Big', 'Bass', 'A', 1.0 ],
        [ 'Big', 'Bass', 'B', 0.0 ],
        [ 'Big', 'Bass', 'C', 0.0 ],
        [ 'Big', 'Drum', 'A', 0.5 ],
        [ 'Big', 'Drum', 'B', 0.5 ],
        [ 'Big', 'Drum', 'C', 0.0 ]]
order_kaka = ['I', 'S', 'G']

xray=[[ 'Low', 'A', 0.885 ],
     [ 'Low', 'B', 0.115 ],
     [ 'Low', 'C', 0.04 ],
     [ 'High', 'A', 0.96 ],
     [ 'High', 'B', 0.96 ],
     [ 'High', 'C', 0.96 ]]
order_xray = ['K', 'G']

In [184]:
f1_vars = order_kaka
f2_vars = order_xray
# first, find all the common variables between the two factors:
common_vars = []
f1 = []
f2 = []
for char in f1_vars:
    if char in f2_vars:
        common_vars.append(char)
    else:
        f1.append(char)

for char in f2_vars:
    if char not in f1_vars:
        f2.append(char)

all_vars = f1+common_vars+f2
num_vars = len(all_vars)
print("comon_var: ", common_vars)
print("f1: ", f1)
print("f2: ", f2)
print("all_vars: ", all_vars)
print("num_vars: ", num_vars)

comon_var:  ['G']
f1:  ['I', 'S']
f2:  ['K']
all_vars:  ['I', 'S', 'G', 'K']
num_vars:  4


In [201]:
#swap common var in original table
f1_str = all_vars[:len(f1_vars)]
f2_str = all_vars[len(f2_vars):]
print(f1_str)
print(f2_str[1:])
print(len(f1_vars))
print(len(f2_vars))

['I', 'S', 'G']
['K']
3
2


In [186]:
index_common_f1 = order_kaka.index(common_vars[0])
index_common_f2 = order_xray.index(common_vars[0])
print(index_common_f1)
print(index_common_f2)

2
1


In [187]:
swapCommon(kaka,index_common_f1,False)
swapCommon(order_kaka,index_common_f1,False)
swapCommon(xray,index_common_f2,True)
swapCommon(order_xray,index_common_f2,True)

27
3
6
2


In [188]:
print(order_xray)

['G', 'K']


In [189]:
print(xray)

[['A', 'Low', 0.885], ['B', 'Low', 0.115], ['C', 'Low', 0.04], ['A', 'High', 0.96], ['B', 'High', 0.96], ['C', 'High', 0.96]]


In [190]:
listI = uniqueElement([kaka[i][order_kaka.index('I')] for i in range(len(kaka))])
listS = uniqueElement([kaka[i][order_kaka.index('S')] for i in range(len(kaka))])
listG = uniqueElement([kaka[i][order_kaka.index('G')] for i in range(len(kaka))])
listK = uniqueElement([xray[i][order_xray.index('K')] for i in range(len(xray))])

In [191]:
print("listI:",listI)
print("listS:",listS)
print("listG:",listG)
print("listK:",listK)

listI: ['Small', 'Medium', 'Big']
listS: ['Guitar', 'Bass', 'Drum']
listG: ['A', 'B', 'C']
listK: ['Low', 'High']


In [192]:
listmerge = []
listmerge.append(listI)
listmerge.append(listS)
listmerge.append(listG)
listmerge.append(listK)
listmerge

[['Small', 'Medium', 'Big'],
 ['Guitar', 'Bass', 'Drum'],
 ['A', 'B', 'C'],
 ['Low', 'High']]

In [199]:
result = parsing_combinationProb(listmerge)
result

[['Small', 'Guitar', 'A', 'Low'],
 ['Small', 'Guitar', 'A', 'High'],
 ['Small', 'Guitar', 'B', 'Low'],
 ['Small', 'Guitar', 'B', 'High'],
 ['Small', 'Guitar', 'C', 'Low'],
 ['Small', 'Guitar', 'C', 'High'],
 ['Small', 'Bass', 'A', 'Low'],
 ['Small', 'Bass', 'A', 'High'],
 ['Small', 'Bass', 'B', 'Low'],
 ['Small', 'Bass', 'B', 'High'],
 ['Small', 'Bass', 'C', 'Low'],
 ['Small', 'Bass', 'C', 'High'],
 ['Small', 'Drum', 'A', 'Low'],
 ['Small', 'Drum', 'A', 'High'],
 ['Small', 'Drum', 'B', 'Low'],
 ['Small', 'Drum', 'B', 'High'],
 ['Small', 'Drum', 'C', 'Low'],
 ['Small', 'Drum', 'C', 'High'],
 ['Medium', 'Guitar', 'A', 'Low'],
 ['Medium', 'Guitar', 'A', 'High'],
 ['Medium', 'Guitar', 'B', 'Low'],
 ['Medium', 'Guitar', 'B', 'High'],
 ['Medium', 'Guitar', 'C', 'Low'],
 ['Medium', 'Guitar', 'C', 'High'],
 ['Medium', 'Bass', 'A', 'Low'],
 ['Medium', 'Bass', 'A', 'High'],
 ['Medium', 'Bass', 'B', 'Low'],
 ['Medium', 'Bass', 'B', 'High'],
 ['Medium', 'Bass', 'C', 'Low'],
 ['Medium', 'Bass', 'C'

In [200]:
for row in range(len(result)):
    key1 = result[row][:len(f1_vars)]
    key2 = result[row][len(f2_vars):]
    #print("key1: ",key1)
    #print("key2: ",key2)
    value1 = get_value(kaka,tuple(key1))
    value2 = get_value(xray,tuple(key2))
    #print("value1: ",value1)
    #print("value2: ",value2)
    product = value1 * value2
    result[row].append(product)
result

[['Small', 'Guitar', 'A', 'Low', 0.0],
 ['Small', 'Guitar', 'A', 'High', 0.0],
 ['Small', 'Guitar', 'B', 'Low', 0.0575],
 ['Small', 'Guitar', 'B', 'High', 0.48],
 ['Small', 'Guitar', 'C', 'Low', 0.02],
 ['Small', 'Guitar', 'C', 'High', 0.48],
 ['Small', 'Bass', 'A', 'Low', 0.0],
 ['Small', 'Bass', 'A', 'High', 0.0],
 ['Small', 'Bass', 'B', 'Low', 0.0],
 ['Small', 'Bass', 'B', 'High', 0.0],
 ['Small', 'Bass', 'C', 'Low', 0.04],
 ['Small', 'Bass', 'C', 'High', 0.96],
 ['Small', 'Drum', 'A', 'Low', 0.0],
 ['Small', 'Drum', 'A', 'High', 0.0],
 ['Small', 'Drum', 'B', 'Low', 0.115],
 ['Small', 'Drum', 'B', 'High', 0.96],
 ['Small', 'Drum', 'C', 'Low', 0.0],
 ['Small', 'Drum', 'C', 'High', 0.0],
 ['Medium', 'Guitar', 'A', 'Low', 0.0],
 ['Medium', 'Guitar', 'A', 'High', 0.0],
 ['Medium', 'Guitar', 'B', 'Low', 0.0],
 ['Medium', 'Guitar', 'B', 'High', 0.0],
 ['Medium', 'Guitar', 'C', 'Low', 0.04],
 ['Medium', 'Guitar', 'C', 'High', 0.96],
 ['Medium', 'Bass', 'A', 'Low', 0.4425],
 ['Medium', 'Bas

In [1]:
from FactorUtilities import *

ft = FactorUtilities()

kaka = [[ 'Small', 'Guitar', 'A', 0.0 ],
        [ 'Small', 'Guitar', 'B', 0.5 ],
        [ 'Small', 'Guitar', 'C', 0.5 ],
        [ 'Small', 'Bass', 'A', 0.0 ],
        [ 'Small', 'Bass', 'B', 0.0 ],
        [ 'Small', 'Bass', 'C', 1.0 ],
        [ 'Small', 'Drum', 'A', 0.0 ],
        [ 'Small', 'Drum', 'B', 1.0 ],
        [ 'Small', 'Drum', 'C', 0.0 ],
        [ 'Medium', 'Guitar', 'A', 0.0 ],
        [ 'Medium', 'Guitar', 'B', 0.0 ],
        [ 'Medium', 'Guitar', 'C', 1.0 ],
        [ 'Medium', 'Bass', 'A', 0.5 ],
        [ 'Medium', 'Bass', 'B', 0.0 ],
        [ 'Medium', 'Bass', 'C', 0.5 ],
        [ 'Medium', 'Drum', 'A', 1.0 ],
        [ 'Medium', 'Drum', 'B', 0.0 ],
        [ 'Medium', 'Drum', 'C', 0.0 ],
        [ 'Big', 'Guitar', 'A', 0.0 ],
        [ 'Big', 'Guitar', 'B', 1.0 ],
        [ 'Big', 'Guitar', 'C', 0.0 ],
        [ 'Big', 'Bass', 'A', 1.0 ],
        [ 'Big', 'Bass', 'B', 0.0 ],
        [ 'Big', 'Bass', 'C', 0.0 ],
        [ 'Big', 'Drum', 'A', 0.5 ],
        [ 'Big', 'Drum', 'B', 0.5 ],
        [ 'Big', 'Drum', 'C', 0.0 ]]
order_kaka = ['I', 'S', 'G']

xray=[[ 'Low', 'A', 0.885 ],
     [ 'Low', 'B', 0.115 ],
     [ 'Low', 'C', 0.04 ],
     [ 'High', 'A', 0.96 ],
     [ 'High', 'B', 0.96 ],
     [ 'High', 'C', 0.96 ]]
order_xray = ['K', 'G']

In [2]:
rl = ft.multiply(kaka,order_kaka,xray,order_xray)
rl

f1:  ['I', 'S']
f2:  ['K']
common_vars:  ['G']
all_vars:  ['I', 'S', 'G', 'K']
index_common_f1:  2
index_common_f2:  1
var1:  I
var1:  S
var1:  G
sub_order_factor2:  ['K']
var2:  K
listInput:  [['Small', 'Medium', 'Big'], ['Guitar', 'Bass', 'Drum'], ['A', 'B', 'C'], ['Low', 'High']]
result:  [['Small', 'Guitar', 'A', 'Low'], ['Small', 'Guitar', 'A', 'High'], ['Small', 'Guitar', 'B', 'Low'], ['Small', 'Guitar', 'B', 'High'], ['Small', 'Guitar', 'C', 'Low'], ['Small', 'Guitar', 'C', 'High'], ['Small', 'Bass', 'A', 'Low'], ['Small', 'Bass', 'A', 'High'], ['Small', 'Bass', 'B', 'Low'], ['Small', 'Bass', 'B', 'High'], ['Small', 'Bass', 'C', 'Low'], ['Small', 'Bass', 'C', 'High'], ['Small', 'Drum', 'A', 'Low'], ['Small', 'Drum', 'A', 'High'], ['Small', 'Drum', 'B', 'Low'], ['Small', 'Drum', 'B', 'High'], ['Small', 'Drum', 'C', 'Low'], ['Small', 'Drum', 'C', 'High'], ['Medium', 'Guitar', 'A', 'Low'], ['Medium', 'Guitar', 'A', 'High'], ['Medium', 'Guitar', 'B', 'Low'], ['Medium', 'Guitar', 'B'

[['Small', 'Guitar', 'A', 'Low', 0.0],
 ['Small', 'Guitar', 'A', 'High', 0.0],
 ['Small', 'Guitar', 'B', 'Low', 0.0575],
 ['Small', 'Guitar', 'B', 'High', 0.48],
 ['Small', 'Guitar', 'C', 'Low', 0.02],
 ['Small', 'Guitar', 'C', 'High', 0.48],
 ['Small', 'Bass', 'A', 'Low', 0.0],
 ['Small', 'Bass', 'A', 'High', 0.0],
 ['Small', 'Bass', 'B', 'Low', 0.0],
 ['Small', 'Bass', 'B', 'High', 0.0],
 ['Small', 'Bass', 'C', 'Low', 0.04],
 ['Small', 'Bass', 'C', 'High', 0.96],
 ['Small', 'Drum', 'A', 'Low', 0.0],
 ['Small', 'Drum', 'A', 'High', 0.0],
 ['Small', 'Drum', 'B', 'Low', 0.115],
 ['Small', 'Drum', 'B', 'High', 0.96],
 ['Small', 'Drum', 'C', 'Low', 0.0],
 ['Small', 'Drum', 'C', 'High', 0.0],
 ['Medium', 'Guitar', 'A', 'Low', 0.0],
 ['Medium', 'Guitar', 'A', 'High', 0.0],
 ['Medium', 'Guitar', 'B', 'Low', 0.0],
 ['Medium', 'Guitar', 'B', 'High', 0.0],
 ['Medium', 'Guitar', 'C', 'Low', 0.04],
 ['Medium', 'Guitar', 'C', 'High', 0.96],
 ['Medium', 'Bass', 'A', 'Low', 0.4425],
 ['Medium', 'Bas

In [1]:
kaka = [[ 'Small', 'Guitar', 'A', 0.0 ],
        [ 'Small', 'Guitar', 'B', 0.5 ],
        [ 'Small', 'Guitar', 'C', 0.5 ],
        [ 'Small', 'Bass', 'A', 0.0 ],
        [ 'Small', 'Bass', 'B', 0.0 ],
        [ 'Small', 'Bass', 'C', 1.0 ],
        [ 'Small', 'Drum', 'A', 0.0 ],
        [ 'Small', 'Drum', 'B', 1.0 ],
        [ 'Small', 'Drum', 'C', 0.0 ],
        [ 'Medium', 'Guitar', 'A', 0.0 ],
        [ 'Medium', 'Guitar', 'B', 0.0 ],
        [ 'Medium', 'Guitar', 'C', 1.0 ],
        [ 'Medium', 'Bass', 'A', 0.5 ],
        [ 'Medium', 'Bass', 'B', 0.0 ],
        [ 'Medium', 'Bass', 'C', 0.5 ],
        [ 'Medium', 'Drum', 'A', 1.0 ],
        [ 'Medium', 'Drum', 'B', 0.0 ],
        [ 'Medium', 'Drum', 'C', 0.0 ],
        [ 'Big', 'Guitar', 'A', 0.0 ],
        [ 'Big', 'Guitar', 'B', 1.0 ],
        [ 'Big', 'Guitar', 'C', 0.0 ],
        [ 'Big', 'Bass', 'A', 1.0 ],
        [ 'Big', 'Bass', 'B', 0.0 ],
        [ 'Big', 'Bass', 'C', 0.0 ],
        [ 'Big', 'Drum', 'A', 0.5 ],
        [ 'Big', 'Drum', 'B', 0.5 ],
        [ 'Big', 'Drum', 'C', 0.0 ]]
        
order_kaka = ['I', 'S', 'G']

In [2]:
import copy 

def removeColumn(factor, order_factor, variable):
    index_column = order_factor.index(variable)
    order_factor.pop(index_column)
    for row in range(len(factor)):
        factor[row].pop(index_column)

def removeValue(factor):
    for row in range(len(factor)):
        factor[row].pop(-1)


In [3]:
new_factor = copy.deepcopy(kaka)
removeColumn(new_factor,order_kaka,'S')
removeValue(new_factor)
new_factor

[['Small', 'A'],
 ['Small', 'B'],
 ['Small', 'C'],
 ['Small', 'A'],
 ['Small', 'B'],
 ['Small', 'C'],
 ['Small', 'A'],
 ['Small', 'B'],
 ['Small', 'C'],
 ['Medium', 'A'],
 ['Medium', 'B'],
 ['Medium', 'C'],
 ['Medium', 'A'],
 ['Medium', 'B'],
 ['Medium', 'C'],
 ['Medium', 'A'],
 ['Medium', 'B'],
 ['Medium', 'C'],
 ['Big', 'A'],
 ['Big', 'B'],
 ['Big', 'C'],
 ['Big', 'A'],
 ['Big', 'B'],
 ['Big', 'C'],
 ['Big', 'A'],
 ['Big', 'B'],
 ['Big', 'C']]

In [4]:
newProbList = []
newValueList = []
count_index = 0
for row in range(len(new_factor)):
    entry = new_factor[row]
    # print("entry: ", entry)
    if entry not in newProbList:
        newProbList.append(entry)
        newValueList.append(kaka[row][-1])
        count_index += 1
    else:
        newValueList[count_index-1] = newValueList[count_index-1] + kaka[row][-1]
    # print("newProbList: ", newProbList)
    # print("newValueList: ", newValueList)
    # print("count_index:", count_index)

for row in range(len(newProbList)):
    newProbList[row].append(newValueList[row])

newProbList


[['Small', 'A', 0.0],
 ['Small', 'B', 0.5],
 ['Small', 'C', 2.5],
 ['Medium', 'A', 0.0],
 ['Medium', 'B', 0.0],
 ['Medium', 'C', 3.0],
 ['Big', 'A', 0.0],
 ['Big', 'B', 1.0],
 ['Big', 'C', 2.0]]

In [5]:
kaka = [[ 'Small', 'Guitar', 'A', 0.0 ],
        [ 'Small', 'Guitar', 'B', 0.5 ],
        [ 'Small', 'Guitar', 'C', 0.5 ],
        [ 'Small', 'Bass', 'A', 0.0 ],
        [ 'Small', 'Bass', 'B', 0.0 ],
        [ 'Small', 'Bass', 'C', 1.0 ],
        [ 'Small', 'Drum', 'A', 0.0 ],
        [ 'Small', 'Drum', 'B', 1.0 ],
        [ 'Small', 'Drum', 'C', 0.0 ],
        [ 'Medium', 'Guitar', 'A', 0.0 ],
        [ 'Medium', 'Guitar', 'B', 0.0 ],
        [ 'Medium', 'Guitar', 'C', 1.0 ],
        [ 'Medium', 'Bass', 'A', 0.5 ],
        [ 'Medium', 'Bass', 'B', 0.0 ],
        [ 'Medium', 'Bass', 'C', 0.5 ],
        [ 'Medium', 'Drum', 'A', 1.0 ],
        [ 'Medium', 'Drum', 'B', 0.0 ],
        [ 'Medium', 'Drum', 'C', 0.0 ],
        [ 'Big', 'Guitar', 'A', 0.0 ],
        [ 'Big', 'Guitar', 'B', 1.0 ],
        [ 'Big', 'Guitar', 'C', 0.0 ],
        [ 'Big', 'Bass', 'A', 1.0 ],
        [ 'Big', 'Bass', 'B', 0.0 ],
        [ 'Big', 'Bass', 'C', 0.0 ],
        [ 'Big', 'Drum', 'A', 0.5 ],
        [ 'Big', 'Drum', 'B', 0.5 ],
        [ 'Big', 'Drum', 'C', 0.0 ]]
order_kaka = ['I', 'S', 'G']

In [6]:
from FactorUtilities import *

ft = FactorUtilities()

so = ft.sum_out(kaka, order_kaka, 'S')
so

[['Small', 'A', 0.0],
 ['Small', 'B', 0.5],
 ['Small', 'C', 2.5],
 ['Medium', 'A', 0.0],
 ['Medium', 'B', 0.0],
 ['Medium', 'C', 3.0],
 ['Big', 'A', 0.0],
 ['Big', 'B', 1.0],
 ['Big', 'C', 2.0]]